# QoS, to make Networks Dependable
<sup> <b>Author:</b> Sarah Samuel (sasamuel@cisco.com) <b>Inputs:</b> Vineet Upendra (vupendra@cisco.com) </sup>

## What is Quality of Service?

Quality of Service (QoS) is the technique of prioritizing traffic flows and providing preferential forwarding for higher-priority packets. The fundamental reason to implement QoS in your network is to provide guaranteed end-to-end traffic forwarding.

Modular QoS provides features that ensure better and more predictable network service by:
* supporting bandwidth allocation
* improving loss characteristics
* avoiding and managing network congestion
* metering network traffic
* setting traffic flow priorities across the network.

This notebook shows you how you can configure QoS on the network with various examples.

NETWORK TOPOLOGY:
![Topology](QoS-topo.png)

## Before You Begin

Before you begin to configure QoS on the simulator, ensure to complete the following prerequisites.
* [Set up simulator](#step1)
* [Add the base configuration](#step2)
* [Design the virtual router topology](#step3)
* [Deploy the configurations](#step4)


## QoS Configurations

After the simulated topology is up and running with the base configurations, we apply the QoS configurations in the below sections:
* [Create a class-map](#step5)
* [Create a policy-map](#step6)
* [Attach the traffic policy to an interface](#step7)
* [Verification of the traffic policy](#step8)
* [Example 1: IP Precedence Marking Configuration](#step9)
* [Example 2: IP DSCP Marking Configuration](#step21)
* [Example 3: QoS Group Marking Configuration](#step10)
* [Example 4: MPLS Experimental Bit Imposition Marking Configuration](#step12)
* [Example 5: MPLS Experimental Topmost Marking Configuration](#step13)

### <a name="step1"></a>Set up Simulator


>First, you'll have to set up the simulator to build your network. The following sets up our python enviroment and the simulator object.

In [1]:
from pathlib import Path
import sys
import os
import logging
import getpass
import string
import random
from pyvxr.vxr import Vxr
print(sys.version)
logging.basicConfig(level=logging.INFO)
sim = Vxr()
# Setting up scratch space for the simulation
sim_dir = '/nobackup/' + getpass.getuser() + '/pyvxr/' + ''.join(random.choices(string.ascii_lowercase + string.digits, k=10))

INFO:pyvxr.vxr:v1.1.1 2021-06-17 13:03 output_dir:vxr.out
INFO:pyvxr.vxr:ba15e2cc5422:/home/vxr/notebooks/Put-Technology-to-Work/QoS


3.8.10 (default, Jun  4 2021, 15:09:15) 
[GCC 7.5.0]


### <a name="step2"></a>Add Base Router Configuration

>This defines and starts up a small two router topology for our lab and a Linux server to use on the same LAN, with SSH to all of the routers.
Takes a minute or five to start. 

In [2]:
ssh_config_str = """
ssh server v2
ssh server netconf
netconf-yang agent ssh
"""

In [3]:
r1_config_str = """
ssh server v2
ssh server netconf
netconf-yang agent ssh
!
hostname R1
!
interface Loopback0
 ipv4 address 10.10.10.10 255.255.255.255
!
interface FourHundredGigE0/0/0/0
 description Connected_to_TREX
 mtu 9216
 ipv4 address 10.0.0.2 255.255.255.0
 no shutdown
!
interface FourHundredGigE0/0/0/1
 description Connected_to_R2
 mtu 9216
 ipv4 address 10.5.7.0 255.255.255.254
 ipv4 unreachables disable
 load-interval 30
 damp
 !
 carrier-delay up 150000 down 0
 load-interval 30
 no shutdown
!
router ospf 10
 router-id 10.10.10.10
 area 0
  interface FourHundredGigE0/0/0/0
  interface FourHundredGigE0/0/0/1
  interface Loopback0
  !
 !
!
mpls ldp
 router-id 10.10.10.10
 interface FourHundredGigE0/0/0/0
 interface FourHundredGigE0/0/0/1
 !
!
 """

In [4]:
r2_config_str = """
ssh server v2
ssh server netconf
netconf-yang agent ssh
!
hostname R2
!
interface Loopback0
 ipv4 address 10.10.10.20 255.255.255.255
!
interface FourHundredGigE0/0/0/0
 description Connected_to_TREX
 mtu 9216
 ipv4 address 10.1.1.2 255.255.255.0
 no shutdown
!
interface FourHundredGigE0/0/0/1
 description Connected_to_R1
 mtu 9216
 ipv4 address 10.5.7.1 255.255.255.254
 ipv4 unreachables disable
 load-interval 30
 dampening
 !
 carrier-delay up 150000 down 0
 load-interval 30
 no shutdown
!
router ospf 10
 router-id 10.10.10.20
 area 0
  interface FourHundredGigE0/0/0/0
  interface FourHundredGigE0/0/0/1
  interface Loopback0
  !
 !
!
mpls ldp
 router-id 10.10.10.20
 interface FourHundredGigE0/0/0/0
 interface FourHundredGigE0/0/0/1
 !
!
"""

In [5]:
%%capture coutput
# To view the cell-output, delete the line above

# Installing pre-requisite modules
!pip install jinja2
!pip install pyats
!pip install genie
!pip install paramiko
!pip install paramiko-expect

### <a name="step3"></a>Design Virtual Topology

>This sets up a small two router topology (spitfire_f).

In [5]:
sys.path.append("../../")
from image_version import *
sys.path.append("..")
from trafficUtils.TrafficGenerator import generate_hipriority_traffic
from trafficUtils.TrafficGenerator import get_ssh_cmd_server

cfg = { 'simulation':
         {'skip_auto_bringup': False,
          'sim_dir': sim_dir,
          'sim_host': 'localhost',
          'sim_rel': '/opt/cisco/vxr2/latest',
          'pyvxr_flags': {'port_file_timeout': 1200 }
         },
        'devices':
        {'R1': {'platform':'spitfire_f-baked',
                'xr_port_redir': [22, 830],
                'data_ports': ['FourH0/0/0/0', 'FourH0/0/0/1', 'FourH0/0/0/2'],
                'linecard_types': ['8201-sys'],
                'xr_config' : r1_config_str,
                'image': sim_image_global,
                'vxr_sim_config': {
                     'shelf': {
                       'ConfigOvxr': ConfigOvxr_global,
                       'ConfigEnableNgdp': ConfigEnableNgdp_global,
                       'ConfigS1SdkVer': ConfigS1SdkVer_global,
                       'ConfigS1NpsuiteVer': ConfigS1NpsuiteVer_global
                     }
                  }
               },
         'R2': {'platform':'spitfire_f-baked',
                'xr_port_redir': [22, 830],
                'data_ports': ['FourH0/0/0/0', 'FourH0/0/0/1', 'FourH0/0/0/2'],
                'linecard_types': ['8201-sys'],
                'xr_config' : r2_config_str,
                'image': sim_image_global,
                'vxr_sim_config': {
                     'shelf': {
                       'ConfigOvxr': ConfigOvxr_global,
                       'ConfigEnableNgdp': ConfigEnableNgdp_global,
                       'ConfigS1SdkVer': ConfigS1SdkVer_global,
                       'ConfigS1NpsuiteVer': ConfigS1NpsuiteVer_global
                     }
                  }
               },
         'trex': {'platform':'linux',
                  'xr_port_redir': [21, 22, 23, 50, 53, 80],
                  'vcpu': 8,
                  'memory': '5GB',
                  'data_ports': ['eth1', 'eth2'],
                  'image': '/opt/cisco/images/linux/centos7_serial.qcow2'
                 }
        },
       'connections':
           {'hubs':
               {
                'hub571':['R1.FourH0/0/0/1', 'R2.FourH0/0/0/1'],
                'hub572':['trex.eth1', 'R1.FourH0/0/0/0'],
                'hub573':['trex.eth2', 'R2.FourH0/0/0/0'] },
           },
      }




### <a name="step4"></a>Deploy Configuration

> Bring up the topology. The bring up can be slow, 5 minutes+

In [6]:
sim.no_image_copy=True
if os.path.exists(sim_dir):
    sim.clean()
    print("Sim clean: Done")
print("Simulation starting. Please wait for the Sim status message. This may take 3-10 minutes.")

try:
    sim.start(cfg)
    status = sim.status()
    print("Sim status: ", status)
except Exception as err:
    print("Sim launch failed (%s)" % str(err))

INFO:pyvxr.vxr:Extracting vxr version from '/opt/cisco/vxr2/latest/setup.sh' file.
INFO:pyvxr.vxr_session:Starting a local bash session for user:vxr


Simulation starting. Please wait for the Sim status message. This may take 3-10 minutes.


INFO:pyvxr.sim:Launch: sim_dir:/nobackup/vxr/pyvxr/wz9562s5gg sim_rel:/opt/cisco/vxr2/latest
INFO:pyvxr.sim:Stopping previous simulation (if any)
INFO:pyvxr.sim:Cleaning previous simulation (if any)
INFO:pyvxr.sim:Starting vxr: 'sim --skiphomecheck -n '
INFO:pyvxr.sim:Vxr up on host localhost
INFO:pyvxr.vxr:Getting port vector files for:R1, R2, trex
INFO:pyvxr.console:R2:wait for XR login prompt (console output captured in vxr.out/logs/console.R2.log)
INFO:pyvxr.console:R1:wait for XR login prompt (console output captured in vxr.out/logs/console.R1.log)
INFO:pyvxr.console:R1:entering new XR username 'cisco', password 'cisco123'
INFO:pyvxr.console:R1:entering XR username 'cisco', password 'cisco123'
INFO:pyvxr.bringup:R1:login successful
INFO:pyvxr.bringup:R1:wait for IOS XR RUN state
INFO:pyvxr.bringup:R1:run state RPs:1 (expected:1) LCs:0 (expected:0)
INFO:pyvxr.bringup:R1:all nodes reached IOS XR RUN state
INFO:pyvxr.console:R1:applying initial XR config (terminal width, etc)
INFO:py

Sim status:  {'localhost': 'running'}


In [11]:
from genie import testbed
from pyats.topology import loader

# A helper function to populate the access details in the pyats yaml file 
def load_yaml_file(yaml_file, *routers):
    import yaml
    console_ports = sim.ports()
    
    y_file = open(yaml_file, 'r')
    y_content = yaml.load(y_file, Loader=yaml.FullLoader)
    y_file.close()
    
    for router in routers:
        y_content['devices'][router]['connections']['cli']['ip'] = console_ports[router]['HostAgent']
        y_content['devices'][router]['connections']['cli']['port'] = console_ports[router]['serial0']
    
    y_file = open(yaml_file, 'w')
    yaml.dump(y_content, y_file)
    y_file.close()
    
    tb = loader.load(yaml_file)
    return tb


In [12]:
tb = load_yaml_file("qos_tb1.yaml", "R1", "R2")

In [13]:
rtr1 = tb.devices['R1']
# Connecting to the Telnet console of R1
out = rtr1.connect()


2021-06-17 13:35:17,700: %UNICON-INFO: +++ R1 logfile /tmp/R1-cli-20210617T133517699.log +++

2021-06-17 13:35:17,703: %UNICON-INFO: +++ Unicon plugin iosxr/spitfire +++
Trying 172.17.0.2...
Connected to 172.17.0.2.
Escape character is '^]'.


2021-06-17 13:35:17,736: %UNICON-INFO: +++ connection to spawn: telnet 172.17.0.2 38186, id: 140421348224112 +++

2021-06-17 13:35:17,738: %UNICON-INFO: connection to R1

RP/0/RP0/CPU0:R1#

2021-06-17 13:35:18,330: %UNICON-INFO: +++ initializing handle +++

2021-06-17 13:35:18,375: %UNICON-INFO: +++ R1 with alias 'cli': executing command 'term length 0' +++
term length 0
Thu Jun 17 13:35:17.851 UTC
RP/0/RP0/CPU0:R1#

2021-06-17 13:35:18,598: %UNICON-INFO: +++ R1 with alias 'cli': executing command 'term width 0' +++
term width 0
Thu Jun 17 13:35:18.064 UTC
RP/0/RP0/CPU0:R1#

2021-06-17 13:35:18,809: %UNICON-INFO: +++ R1 with alias 'cli': executing command 'show version' +++
show version
Thu Jun 17 13:35:18.278 UTC
Cisco IOS XR Software, Version 

### Start the network traffic

In [15]:
sys.path.append("..")

# Import the function that generates high priority traffic
from trafficUtils.TrafficGenerator import generate_hipriority_traffic
ports = sim.ports()
trexipaddress = str(ports['trex']['HostAgent'])
trexport = str(ports['trex']['xr_redir22'])
client1, client2, interact1, interact2 = generate_hipriority_traffic (trexipaddress, trexport)

INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_7.4)
INFO:paramiko.transport:Authentication (publickey) failed.
INFO:paramiko.transport:Authentication (password) successful!
INFO:paramiko.transport.sftp:[chan 0] Opened sftp connection (server version 3)
INFO:paramiko.transport.sftp:[chan 0] sftp session closed.


Last login: Thu Jun 17 13:06:55 2021
[root@localhost ~]# ifconfig eth1 10.0.0.1 netmask 255.255.255.0 up; ifconfig et h2 10.1.1.1 netmask 255.255.255.0 up
[root@localhost ~]# cd /opt/cisco/trex/latest/
[root@localhost latest]# ./t-rex-64 -i
Starting Scapy server....... Scapy server is started
Trying to bind to vfio-pci ...
Trying to compile and bind to igb_uio ...
ERROR: We don't have precompiled igb_uio.ko module for your kernel version.
Will try compiling automatically...
Success.

/usr/bin/python3 dpdk_nic_bind.py --bind=igb_uio 0000:00:04.0 0000:00:05.0 
The ports are bound/configured.
Starting  TRex v2.89 please wait  ... 
 set driver name net_virtio 
 driver capability  :
 set dpdk queues mode to ONE_QUE 
 Number of ports found: 2
zmq publisher at: tcp://*:4500
 wait 1 sec 

INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_7.4)
INFO:paramiko.transport:Authentication (publickey) failed.
INFO:paramiko.transport:Authentication (password) successful!


Last login: Thu Jun 17 13:35:31 2021 from gateway
[root@localhost ~]# cd /opt/cisco/trex/latest/
[root@localhost latest]# ./trex-console
start -f stl/dscp_traffic1.py -d 1h -m 100pps -p 0

Using 'python3' as Python interpeter


Connecting to RPC server on localhost:4501                   [SUCCESS]


Connecting to publisher server on localhost:4500             [SUCCESS]


Acquiring ports [0, 1]:                                      [SUCCESS]


Server Info:

Server version:   v2.89 @ STL
Server mode:      Stateless
Server CPU:       1 x VXR
Ports count:      2 x 100.0Gbps @ Virtio network device	

-=TRex Console v3.0=-

Type 'help' or '?' for supported actions

trex>start -f stl/dscp_traffic1.py -d 1h -m 100pps -p 0


## <a name="step5"></a>Create a class map

>First, create the class-map by executing the below steps.

In [16]:
# The below steps configures the class-map.
out = rtr1.configure('''
class-map match-any class1
 match dscp ef
 end-class-map
''')


2021-06-17 13:35:50,896: %UNICON-INFO: +++ R1 with alias 'cli': configure +++
configure terminal
RP/0/RP0/CPU0:R1(config)#
RP/0/RP0/CPU0:R1(config)#class-map match-any class1
RP/0/RP0/CPU0:R1(config-cmap)# match dscp ef
RP/0/RP0/CPU0:R1(config-cmap)# end-class-map
RP/0/RP0/CPU0:R1(config)#commit
RP/0/RP0/CPU0:R1(config)#end
RP/0/RP0/CPU0:R1#


>Verify the class-map with show commands.

In [17]:
out = rtr1.execute('show class-map class1')


2021-06-17 13:35:54,632: %UNICON-INFO: +++ R1 with alias 'cli': executing command 'show class-map class1' +++
show class-map class1
1) ClassMap: class1    Type: qos 
    Referenced by 0 Policymaps 

RP/0/RP0/CPU0:R1#


## <a name="step6"></a>Create a policy-map

In [18]:
# This step configures the policy-map using the class-map created in the previous step.
out = rtr1.configure('''
policy-map policy1
 class class1
  set qos-group 4
 class class-default 
 end-policy-map
''')


2021-06-17 13:36:01,080: %UNICON-INFO: +++ R1 with alias 'cli': configure +++
configure terminal
RP/0/RP0/CPU0:R1(config)#
RP/0/RP0/CPU0:R1(config)#policy-map policy1
RP/0/RP0/CPU0:R1(config-pmap)# class class1
RP/0/RP0/CPU0:R1(config-pmap-c)#  set qos-group 4
RP/0/RP0/CPU0:R1(config-pmap-c)# class class-default 
RP/0/RP0/CPU0:R1(config-pmap-c)# end-policy-map
RP/0/RP0/CPU0:R1(config)#commit
RP/0/RP0/CPU0:R1(config)#end
RP/0/RP0/CPU0:R1#


> Verify the policy-map with show commands.

In [55]:
out = rtr1.execute('show policy-map policy1')


2021-06-17 15:30:46,299: %UNICON-INFO: +++ R1 with alias 'cli': executing command 'show policy-map' +++
show policy-map
policy-map policy-qg
 class class3
  set qos-group 1
 ! 
 class class-default
 ! 
 end-policy-map
! 
policy-map policy1
 class class1
  set qos-group 4
 ! 
 class class-default
 ! 
 end-policy-map
! 
RP/0/RP0/CPU0:R1#


## <a name="step7"></a>Apply the traffic policy to an interface

In [53]:
# Applying traffic policy to an interface in ingress direction.
# Note that policy can be applied on the interface in the ingress and egress direction.
out = rtr1.configure('''
interface FourHundredGigE0/0/0/0
 service-policy input policy1
''')


2021-06-17 14:31:21,439: %UNICON-INFO: +++ R1 with alias 'cli': configure +++
configure terminal
RP/0/RP0/CPU0:R1(config)#
RP/0/RP0/CPU0:R1(config)#interface FourHundredGigE0/0/0/0
RP/0/RP0/CPU0:R1(config-if)# service-policy input policy1
RP/0/RP0/CPU0:R1(config-if)#commit
RP/0/RP0/CPU0:R1(config-if)#end
RP/0/RP0/CPU0:R1#


## <a name="step8"></a>Verification of the traffic policy

In [20]:
# now display the policy applied on the interface.
out = rtr1.execute('show qos interface FourHundredGigE0/0/0/0 input')


2021-06-17 13:38:04,018: %UNICON-INFO: +++ R1 with alias 'cli': executing command 'show qos interface FourHundredGigE0/0/0/0 input' +++
show qos interface FourHundredGigE0/0/0/0 input
NOTE:- Configured values are displayed within parentheses
Interface FourHundredGigE0/0/0/0 ifh 0xf000138  -- input policy
NPU Id:                        0
Total number of classes:       2
Interface Bandwidth:           400000000 kbps
Policy Name:                   policy1
Accounting Type:               Layer1 (Include Layer 1 encapsulation and above)
------------------------------------------------------------------------------
Level1 Class                             =   class1
New qos group                            =   4
Policer not configured for this class

Level1 Class                             =   class-default
Policer not configured for this class
RP/0/RP0/CPU0:R1#


In [21]:
# Check the traffic on the port
out = rtr1.execute('show interface FourHundredGigE0/0/0/0 accounting')
# Policy-map counter updates are v slow. To see the counters incrementing, replay this cell a few times 
out = rtr1.execute('show policy-map interface FourHundredGigE0/0/0/0 input')


2021-06-17 13:38:06,582: %UNICON-INFO: +++ R1 with alias 'cli': executing command 'show interface FourHundredGigE0/0/0/0 accounting' +++
show interface FourHundredGigE0/0/0/0 accounting
FourHundredGigE0/0/0/0
  Protocol              Pkts In         Chars In     Pkts Out        Chars Out
  IPV4_UNICAST            14503                0            0                0
  ARP                         4              240            2               84


RP/0/RP0/CPU0:R1#

2021-06-17 13:38:06,820: %UNICON-INFO: +++ R1 with alias 'cli': executing command 'show policy-map interface FourHundredGigE0/0/0/0 input' +++
show policy-map interface FourHundredGigE0/0/0/0 input

FourHundredGigE0/0/0/0 input: policy1

Class class1
  Classification statistics          (packets/bytes)     (rate - kbps)
    Matched             :                8092/0                    0
    Transmitted         :                8092/0                    0
    Total Dropped       :                   0/0                    0
Cla

## <a name="step9"></a>Example 1: IP Precedence Marking Configuration

> In the previous sections, you have already configured an input service-policy on the input interface **FourHundredGigE0/0/0/0** to set the qos-group value to 4 for all incoming traffic packets which have the **DSCP** bits set to **EF** (Expedited Forwarding). 

> Play the next cell to configure an output service policy named **out-policy1** on the output interface **FourHundredGigE0/0/0/1**. This service-policy matches all the outgoing trafic packets that have qos-group value set to 4 and sets the IP precedence bit to 1 for the matched packets. 

In [22]:
out = rtr1.configure('''
class-map match-any out-class1
 match qos-group 4
 end-class-map

policy-map out-policy1
 class out-class1
  set precedence 1

interface FourHundredGigE0/0/0/1
 service-policy output out-policy1
''')


2021-06-17 13:38:28,755: %UNICON-INFO: +++ R1 with alias 'cli': configure +++
configure terminal
RP/0/RP0/CPU0:R1(config)#
RP/0/RP0/CPU0:R1(config)#class-map match-any out-class1
RP/0/RP0/CPU0:R1(config-cmap)# match qos-group 4
RP/0/RP0/CPU0:R1(config-cmap)# end-class-map
RP/0/RP0/CPU0:R1(config)#
RP/0/RP0/CPU0:R1(config)#policy-map out-policy1
RP/0/RP0/CPU0:R1(config-pmap)# class out-class1
RP/0/RP0/CPU0:R1(config-pmap-c)#  set precedence 1
RP/0/RP0/CPU0:R1(config-pmap-c)#
RP/0/RP0/CPU0:R1(config-pmap-c)#interface FourHundredGigE0/0/0/1
RP/0/RP0/CPU0:R1(config-if)# service-policy output out-policy1
RP/0/RP0/CPU0:R1(config-if)#commit
RP/0/RP0/CPU0:R1(config-if)#end
RP/0/RP0/CPU0:R1#


> The egress packets on interface **FourHundredGigE0/0/0/1**  matches the class-map **out-class1** in the service-policy **out-policy1**. You can view the policy-map details and counters by playing the next cell. Note that the policy-map counter updates are slow. To see the policy-map counters incrementing, replay this cell a few times

In [29]:
# now display the policy applied on the interface.
out = rtr1.execute('show qos interface FourHundredGigE0/0/0/1 output')

# Other useful show commands
out = rtr1.execute('show interface FourHundredGigE0/0/0/1 accounting')
# Policy-map counter updates are v slow. To see the counters incrementing, replay this cell a few times 
out = rtr1.execute('show policy-map interface FourHundredGigE0/0/0/1 output')



2021-06-17 13:39:30,187: %UNICON-INFO: +++ R1 with alias 'cli': executing command 'show qos interface FourHundredGigE0/0/0/1 output' +++
show qos interface FourHundredGigE0/0/0/1 output
NOTE:- Configured values are displayed within parentheses
Interface FourHundredGigE0/0/0/1 ifh 0xf0001a0  -- output policy
NPU Id:                        0
Total number of classes:       2
Interface Bandwidth:           400000000 kbps
Policy Name:                   out-policy1
VOQ Base:                      0
Accounting Type:               Layer1 (Include Layer 1 encapsulation and above)
VOQ Mode:                      8
Shared Counter Mode:           1
------------------------------------------------------------------------------
Level1 Class                             =   out-class1
New prec                                 =   1

Level1 Class                             =   class-default
RP/0/RP0/CPU0:R1#

2021-06-17 13:39:30,637: %UNICON-INFO: +++ R1 with alias 'cli': executing command 'show interfa

In [30]:
# Remove the previous commit in order to configure more policies.
out = rtr1.execute('rollback configuration last 1')


2021-06-17 13:39:35,396: %UNICON-INFO: +++ R1 with alias 'cli': executing command 'rollback configuration last 1' +++
rollback configuration last 1

Loading Rollback Changes.
Loaded Rollback Changes in 1 sec 
Committing.
5 items committed in 1 sec 
Updating.
Updated Commit database in 1 sec 
Configuration successfully rolled back 1 commits.
RP/0/RP0/CPU0:R1#


## <a name="step21"></a>Example 2: IP DSCP Marking Configuration

> Play the next cell to configure an output service policy named **policy-dscp** on the output interface **FourHundredGigE0/0/0/1**. This service-policy matches all the outgoing trafic packets that have qos-group value set to 4 and sets the IP DSCP value to 5 for the matched packets. 

In [31]:
out = rtr1.configure('''
class-map match-any out-class1
 match qos-group 4
 end-class-map

policy-map policy-dscp
 class out-class1
  set dscp 5
 class class-default
 end-policy-map

interface FourHundredGigE0/0/0/1
 service-policy output policy-dscp
''')


2021-06-17 13:40:04,211: %UNICON-INFO: +++ R1 with alias 'cli': configure +++
configure terminal
RP/0/RP0/CPU0:R1(config)#
RP/0/RP0/CPU0:R1(config)#class-map match-any out-class1
RP/0/RP0/CPU0:R1(config-cmap)# match qos-group 4
RP/0/RP0/CPU0:R1(config-cmap)# end-class-map
RP/0/RP0/CPU0:R1(config)#
RP/0/RP0/CPU0:R1(config)#policy-map policy-dscp
RP/0/RP0/CPU0:R1(config-pmap)# class out-class1
RP/0/RP0/CPU0:R1(config-pmap-c)#  set dscp 5
RP/0/RP0/CPU0:R1(config-pmap-c)# class class-default
RP/0/RP0/CPU0:R1(config-pmap-c)# end-policy-map
RP/0/RP0/CPU0:R1(config)#
RP/0/RP0/CPU0:R1(config)#interface FourHundredGigE0/0/0/1
RP/0/RP0/CPU0:R1(config-if)# service-policy output policy-dscp
RP/0/RP0/CPU0:R1(config-if)#commit
RP/0/RP0/CPU0:R1(config-if)#end
RP/0/RP0/CPU0:R1#


> The egress traffic packets on interface **FourHundredGigE0/0/0/1**  matches the class-map **out-class1** in the service-policy **policy-dscp**, because the input service-policy on the input interface ie **FourHundredGigE0/0/0/0** sets the qos-group value to 4 for the incoming packets.

In [35]:
# now display the policy applied on the interface.
out = rtr1.execute('show qos interface FourHundredGigE0/0/0/1 output')

# Other useful show commands
out = rtr1.execute('show interface FourHundredGigE0/0/0/1 accounting')
# Policy-map counter updates are v slow. To see the counters incrementing, replay this cell a few times 
out = rtr1.execute('show policy-map interface FourHundredGigE0/0/0/1 output')


2021-06-17 13:41:02,775: %UNICON-INFO: +++ R1 with alias 'cli': executing command 'show qos interface FourHundredGigE0/0/0/1 output' +++
show qos interface FourHundredGigE0/0/0/1 output
NOTE:- Configured values are displayed within parentheses
Interface FourHundredGigE0/0/0/1 ifh 0xf0001a0  -- output policy
NPU Id:                        0
Total number of classes:       2
Interface Bandwidth:           400000000 kbps
Policy Name:                   policy-dscp
VOQ Base:                      0
Accounting Type:               Layer1 (Include Layer 1 encapsulation and above)
VOQ Mode:                      8
Shared Counter Mode:           1
------------------------------------------------------------------------------
Level1 Class                             =   out-class1
New dscp                                 =   5

Level1 Class                             =   class-default
RP/0/RP0/CPU0:R1#

2021-06-17 13:41:03,012: %UNICON-INFO: +++ R1 with alias 'cli': executing command 'show interfa

In [36]:
# Remove the previous commit in order to configure more policies.
out = rtr1.execute('rollback configuration last 1')


2021-06-17 13:41:06,912: %UNICON-INFO: +++ R1 with alias 'cli': executing command 'rollback configuration last 1' +++
rollback configuration last 1

Loading Rollback Changes.
Loaded Rollback Changes in 1 sec 
Committing.
5 items committed in 1 sec (4)items/sec
Updating.
Updated Commit database in 1 sec 
Configuration successfully rolled back 1 commits.
RP/0/RP0/CPU0:R1#


## <a name="step10"></a>Example 3: QoS Group Marking Configuration

> In this example, you will unconfigure the input service-policy **policy1** and then configure another input service policy named **policy-qg** on the interface **FourHundredGigE0/0/0/0**. The qos-group value is set to 1 for all incoming traffic packets that match **class3**. 

In [38]:
out = rtr1.configure('''
class-map match-any class3
 match protocol ipv4
 match access-group ipv4 101
 end-class-map

policy-map policy-qg
 class class3
  set qos-group 1
  end-policy-map

interface FourHundredGigE0/0/0/0
 no service-policy input policy1
 service-policy input policy-qg
''')


2021-06-17 13:43:16,631: %UNICON-INFO: +++ R1 with alias 'cli': configure +++
configure terminal
RP/0/RP0/CPU0:R1(config)#
RP/0/RP0/CPU0:R1(config)#class-map match-any class3
RP/0/RP0/CPU0:R1(config-cmap)# match protocol ipv4
RP/0/RP0/CPU0:R1(config-cmap)# match access-group ipv4 101
RP/0/RP0/CPU0:R1(config-cmap)# end-class-map
RP/0/RP0/CPU0:R1(config)#
RP/0/RP0/CPU0:R1(config)#policy-map policy-qg
RP/0/RP0/CPU0:R1(config-pmap)# class class3
RP/0/RP0/CPU0:R1(config-pmap-c)#  set qos-group 1
RP/0/RP0/CPU0:R1(config-pmap-c)#  end-policy-map
RP/0/RP0/CPU0:R1(config)#
RP/0/RP0/CPU0:R1(config)#interface FourHundredGigE0/0/0/0
RP/0/RP0/CPU0:R1(config-if)# no service-policy input policy1
RP/0/RP0/CPU0:R1(config-if)# service-policy input policy-qg
RP/0/RP0/CPU0:R1(config-if)#commit
RP/0/RP0/CPU0:R1(config-if)#end
RP/0/RP0/CPU0:R1#


In [39]:
# now Display the policy applied on the interface.
out = rtr1.execute('show qos interface FourHundredGigE0/0/0/0 input')

# Other useful show commands
out = rtr1.execute('show interface FourHundredGigE0/0/0/0 accounting')
# Policy-map counter updates are v slow. To see the counters incrementing, replay this cell a few times 
out = rtr1.execute('show policy-map interface FourHundredGigE0/0/0/0 input')


2021-06-17 13:44:19,438: %UNICON-INFO: +++ R1 with alias 'cli': executing command 'show qos interface FourHundredGigE0/0/0/0 input' +++
show qos interface FourHundredGigE0/0/0/0 input
NOTE:- Configured values are displayed within parentheses
Interface FourHundredGigE0/0/0/0 ifh 0xf000138  -- input policy
NPU Id:                        0
Total number of classes:       2
Interface Bandwidth:           400000000 kbps
Policy Name:                   policy-qg
Accounting Type:               Layer1 (Include Layer 1 encapsulation and above)
------------------------------------------------------------------------------
Level1 Class                             =   class3
New qos group                            =   1
Policer not configured for this class

Level1 Class                             =   class-default
Policer not configured for this class
RP/0/RP0/CPU0:R1#

2021-06-17 13:44:19,675: %UNICON-INFO: +++ R1 with alias 'cli': executing command 'show interface FourHundredGigE0/0/0/0 accoun

> Unconfigure the service-policy using the "no" form of the service-policy command instead of **rollback configuration**.

In [40]:
# Remove the policy-map using "no" of the command
out = rtr1.configure('''
interface FourHundredGigE0/0/0/0
no service-policy input policy-qg
''')


2021-06-17 13:45:16,297: %UNICON-INFO: +++ R1 with alias 'cli': configure +++
configure terminal
RP/0/RP0/CPU0:R1(config)#
RP/0/RP0/CPU0:R1(config)#interface FourHundredGigE0/0/0/0
RP/0/RP0/CPU0:R1(config-if)#no service-policy input policy-qg
RP/0/RP0/CPU0:R1(config-if)#commit
RP/0/RP0/CPU0:R1(config-if)#end
RP/0/RP0/CPU0:R1#


## <a name="step12"></a>Example 4: MPLS Experimental Bit Imposition Marking Configuration

>In this example, you will configure an input service policy **policy-mex-imp**. This service policy is associated to a class-map **class3** through the use of the class command. This service-policy sets the MPLS EXP bits of the MPLS labels to 1 for the matched incoming packets.

In [41]:
out = rtr1.configure('''
class-map match-any class3
 match protocol ipv4
 match access-group ipv4 101
 end-class-map

policy-map policy-mex-imp
 class class3
  set mpls exp imposition 1
  end-policy-map

interface FourHundredGigE0/0/0/0
 service-policy input policy-mex-imp
''')


2021-06-17 13:46:04,818: %UNICON-INFO: +++ R1 with alias 'cli': configure +++
configure terminal
RP/0/RP0/CPU0:R1(config)#
RP/0/RP0/CPU0:R1(config)#class-map match-any class3
RP/0/RP0/CPU0:R1(config-cmap)# match protocol ipv4
RP/0/RP0/CPU0:R1(config-cmap)# match access-group ipv4 101
RP/0/RP0/CPU0:R1(config-cmap)# end-class-map
RP/0/RP0/CPU0:R1(config)#
RP/0/RP0/CPU0:R1(config)#policy-map policy-mex-imp
RP/0/RP0/CPU0:R1(config-pmap)# class class3
RP/0/RP0/CPU0:R1(config-pmap-c)#  set mpls exp imposition 1
RP/0/RP0/CPU0:R1(config-pmap-c)#  end-policy-map
RP/0/RP0/CPU0:R1(config)#
RP/0/RP0/CPU0:R1(config)#interface FourHundredGigE0/0/0/0
RP/0/RP0/CPU0:R1(config-if)# service-policy input policy-mex-imp
RP/0/RP0/CPU0:R1(config-if)#commit
RP/0/RP0/CPU0:R1(config-if)#end
RP/0/RP0/CPU0:R1#


In [45]:
# now display the policy applied on the interface.
out = rtr1.execute('show qos interface FourHundredGigE0/0/0/0 input')

# Other useful show commands
out = rtr1.execute('show interface FourHundredGigE0/0/0/0 accounting')
# Policy-map counter updates are v slow. To see the counters incrementing, replay this cell a few times 
out = rtr1.execute('show policy-map interface FourHundredGigE0/0/0/0 input')


2021-06-17 13:47:00,064: %UNICON-INFO: +++ R1 with alias 'cli': executing command 'show qos interface FourHundredGigE0/0/0/0 input' +++
show qos interface FourHundredGigE0/0/0/0 input
NOTE:- Configured values are displayed within parentheses
Interface FourHundredGigE0/0/0/0 ifh 0xf000138  -- input policy
NPU Id:                        0
Total number of classes:       2
Interface Bandwidth:           400000000 kbps
Policy Name:                   policy-mex-imp
Accounting Type:               Layer1 (Include Layer 1 encapsulation and above)
------------------------------------------------------------------------------
Level1 Class                             =   class3
New imposition exp                       =   1
Policer not configured for this class

Level1 Class                             =   class-default
Policer not configured for this class
RP/0/RP0/CPU0:R1#

2021-06-17 13:47:00,301: %UNICON-INFO: +++ R1 with alias 'cli': executing command 'show interface FourHundredGigE0/0/0/0 a

In [46]:
# Remove the previous commit in order to configure more policies.
out = rtr1.execute('rollback configuration last 1')


2021-06-17 13:47:05,248: %UNICON-INFO: +++ R1 with alias 'cli': executing command 'rollback configuration last 1' +++
rollback configuration last 1

Loading Rollback Changes.
Loaded Rollback Changes in 1 sec 
Committing.
5 items committed in 1 sec (4)items/sec
Updating.
Updated Commit database in 1 sec 
Configuration successfully rolled back 1 commits.
RP/0/RP0/CPU0:R1#


## <a name="step13"></a>Example 5: MPLS Experimental Topmost Marking Configuration

> Play this cell to configure a service policy **policy-mex-top**. This service policy is associated to a class map **class-mex-top** through the use of the class command, and then the service policy is applied in the output direction on interface **FourHundredGigE0/0/0/1**. The MPLS EXP bits on the topmost label are set to 1 for all matched traffic packets.

In [47]:
out = rtr1.configure('''
class-map match-any class-mex-top
 match mpls exp topmost 1
 end-class-map

policy-map policy-mex-top
 class class-mex-top
  set mpls exp topmost 2
  end-policy-map

interface FourHundredGigE0/0/0/1
 service-policy output policy-mex-top
''')


2021-06-17 13:51:56,712: %UNICON-INFO: +++ R1 with alias 'cli': configure +++
configure terminal
RP/0/RP0/CPU0:R1(config)#
RP/0/RP0/CPU0:R1(config)#class-map match-any class-mex-top
RP/0/RP0/CPU0:R1(config-cmap)# match mpls exp topmost 1
RP/0/RP0/CPU0:R1(config-cmap)# end-class-map
RP/0/RP0/CPU0:R1(config)#
RP/0/RP0/CPU0:R1(config)#policy-map policy-mex-top
RP/0/RP0/CPU0:R1(config-pmap)# class class-mex-top
RP/0/RP0/CPU0:R1(config-pmap-c)#  set mpls exp topmost 2
RP/0/RP0/CPU0:R1(config-pmap-c)#  end-policy-map
RP/0/RP0/CPU0:R1(config)#
RP/0/RP0/CPU0:R1(config)#interface FourHundredGigE0/0/0/1
RP/0/RP0/CPU0:R1(config-if)# service-policy output policy-mex-top
RP/0/RP0/CPU0:R1(config-if)#commit
RP/0/RP0/CPU0:R1(config-if)#end
RP/0/RP0/CPU0:R1#


In [51]:
# now display the policy applied on the interface.
out = rtr1.execute('show qos interface FourHundredGigE0/0/0/1 output')

# Other useful show commands
out = rtr1.execute('show interface FourHundredGigE0/0/0/1 accounting')
# Policy-map counter updates are v slow. To see the counters incrementing, replay this cell a few times 
out = rtr1.execute('show policy-map interface FourHundredGigE0/0/0/1 output')


2021-06-17 13:54:14,566: %UNICON-INFO: +++ R1 with alias 'cli': executing command 'show qos interface FourHundredGigE0/0/0/1 output' +++
show qos interface FourHundredGigE0/0/0/1 output
NOTE:- Configured values are displayed within parentheses
Interface FourHundredGigE0/0/0/1 ifh 0xf0001a0  -- output policy
NPU Id:                        0
Total number of classes:       2
Interface Bandwidth:           400000000 kbps
Policy Name:                   policy-mex-top
VOQ Base:                      0
Accounting Type:               Layer1 (Include Layer 1 encapsulation and above)
VOQ Mode:                      8
Shared Counter Mode:           1
------------------------------------------------------------------------------
Level1 Class                             =   class-mex-top
New topmost exp                          =   2

Level1 Class                             =   class-default
RP/0/RP0/CPU0:R1#

2021-06-17 13:54:14,800: %UNICON-INFO: +++ R1 with alias 'cli': executing command 'show i

In [52]:
# Remove the previous commit in order to configure more policies.
out = rtr1.execute('rollback configuration last 1')


2021-06-17 13:54:18,968: %UNICON-INFO: +++ R1 with alias 'cli': executing command 'rollback configuration last 1' +++
rollback configuration last 1

Loading Rollback Changes.
Loaded Rollback Changes in 1 sec 
Committing.
5 items committed in 1 sec 
Updating.
Updated Commit database in 1 sec 
Configuration successfully rolled back 1 commits.
RP/0/RP0/CPU0:R1#


>Once you finished you can shutdown the topology and clean up.

In [9]:
# Close the sim
sim.stop()
sim.clean()

INFO:pyvxr.vxr:Stopping sim on host localhost (dir /nobackup/vxr/pyvxr/5v0n02yijx)
INFO:pyvxr.sim:Stopping previous simulation (if any)
INFO:pyvxr.vxr:Cleaning sim on host localhost (dir /nobackup/vxr/pyvxr/5v0n02yijx)
INFO:pyvxr.sim:Stopping previous simulation (if any)
INFO:pyvxr.sim:Cleaning previous simulation (if any)


In [10]:
# Clean up sim scratch space
import shutil
shutil.rmtree(sim_dir)